In [1]:
# Understanding autoencoders via a simple example

## generate the same dataset

In [41]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l


In [42]:
# I feel that there should be a relationship between columns in dataset.  not just random. Let's see if I'm right?
## Comment: after figuring this in my mind, I realized it is always true to have a realtionship between features
## in simple words, yeah they must have a relationship...!

In [43]:
first_col_random = torch.rand(10,1) # producing a random column
features = torch.cat((first_col_random, 2*first_col_random), 1) # that is how a relationship between cols 
# are introduced here in my example.

In [44]:
# now, I want to have more columns. each colmn is multiplied from first column
first_col_random = torch.rand(1000,1) 
features = first_col_random # initial values for the fist col
for i in range(2,5):
    features = torch.cat((features, i*first_col_random), 1) # creation of other colmn

In [45]:
# see? they are multipied from the first col
features

tensor([[0.8258, 1.6516, 2.4774, 3.3032],
        [0.0995, 0.1989, 0.2984, 0.3978],
        [0.3325, 0.6649, 0.9974, 1.3299],
        ...,
        [0.8316, 1.6632, 2.4947, 3.3263],
        [0.0720, 0.1440, 0.2160, 0.2880],
        [0.7045, 1.4091, 2.1136, 2.8181]])

In [46]:
# first_col_random

In [53]:
from matplotlib.pyplot import plot as plt

In [54]:
plt.imshow(features)

AttributeError: 'function' object has no attribute 'imshow'

In [47]:
# in auto encoder we want a reperesentation of our data. The best representation here is the one
# shows that colmns are multiplied from the first one (abvious). But this one seems like to be 
# an unsupervised learning. But no. Let's think differently. We want a representation x_ of our data x
# so, I guess now it is a supervised learning. We only need to define ground truth as y = x and
# come up with wieghts and biases (through training) to best fit this.

# based on the explanation above, let's figure this out:
labels = features

In [48]:
features.shape, labels.shape

(torch.Size([1000, 4]), torch.Size([1000, 4]))

In [49]:
features[0], labels[0]

(tensor([0.8258, 1.6516, 2.4774, 3.3032]),
 tensor([0.8258, 1.6516, 2.4774, 3.3032]))

## Reading the data set

In [50]:
def load_array(data_arrays, batch_size, is_train=True): #@save
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


In [17]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [18]:
data_iter

In [19]:
# it is import to use iterator for that
next(iter(data_iter))

[tensor([[0.9713, 1.9426, 2.9138, 3.8851],
         [0.3541, 0.7083, 1.0624, 1.4165],
         [0.4076, 0.8151, 1.2227, 1.6302],
         [0.5732, 1.1463, 1.7195, 2.2926],
         [0.5761, 1.1522, 1.7283, 2.3044],
         [0.6568, 1.3137, 1.9705, 2.6273],
         [0.2455, 0.4910, 0.7366, 0.9821],
         [0.7789, 1.5577, 2.3366, 3.1154],
         [0.5109, 1.0219, 1.5328, 2.0437],
         [0.5673, 1.1347, 1.7020, 2.2694]]),
 tensor([[0.9713, 1.9426, 2.9138, 3.8851],
         [0.3541, 0.7083, 1.0624, 1.4165],
         [0.4076, 0.8151, 1.2227, 1.6302],
         [0.5732, 1.1463, 1.7195, 2.2926],
         [0.5761, 1.1522, 1.7283, 2.3044],
         [0.6568, 1.3137, 1.9705, 2.6273],
         [0.2455, 0.4910, 0.7366, 0.9821],
         [0.7789, 1.5577, 2.3366, 3.1154],
         [0.5109, 1.0219, 1.5328, 2.0437],
         [0.5673, 1.1347, 1.7020, 2.2694]])]

## Defining the Model

In [20]:
# The bottleneck here is only of dim 2

In [21]:
import torch.nn as nn
import torch as T

class Net(T.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = T.nn.Linear(4, 2)  # 6-(10-10)-3
        self.oupt = T.nn.Linear(2, 4)
    

        T.nn.init.xavier_uniform_(self.hid1.weight)
        T.nn.init.zeros_(self.hid1.bias)
        T.nn.init.xavier_uniform_(self.oupt.weight)
        T.nn.init.zeros_(self.oupt.bias)

    def forward(self, x):
        z = T.tanh(self.hid1(x))
        z = self.oupt(z)  # no softmax: CrossEntropyLoss() 
        
        return z

In [22]:
net = Net()

In [23]:
# model:
net

Net(
  (hid1): Linear(in_features=4, out_features=2, bias=True)
  (oupt): Linear(in_features=2, out_features=4, bias=True)
)

## Initializing Model Parameters

In [24]:
# net[0] has accesss to the first layer of net

# net[0].weight.data.normal_(0, 0.01)
# net[0].bias.data.fill_(0)

## Defining the Loss Function

In [25]:
loss = nn.MSELoss()

In [26]:
# testing loss function:
X = torch.tensor([[-1.2373,  0.3100, -1.4755, -1.1790]])
y = X
# net(X)
loss(net(X), y)

tensor(2.0736, grad_fn=<MseLossBackward0>)

## Defining the Optimization Algorithm


In [28]:
# Minibatch stochastic gradient descent is a standard tool for optimizing neural networks and thus
# PyTorch supports it alongside a number of variations on this algorithm in the optim module. When
# we instantiate an SGD instance, we will specify the parameters to optimize over (obtainable from
# our net via net.parameters()), with a dictionary of hyperparameters required by our optimization
# algorithm. Minibatch stochastic gradient descent just requires that we set the value lr, which is
# set to 0.03 here.

trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [29]:
net.parameters()

<generator object Module.parameters at 0x7f4db16ceba0>

In [30]:
list(net.parameters())

[Parameter containing:
 tensor([[-0.3413,  0.6577,  0.2600,  0.1821],
         [-0.5019, -0.3456, -0.9326,  0.5788]], requires_grad=True),
 Parameter containing:
 tensor([0., 0.], requires_grad=True),
 Parameter containing:
 tensor([[ 0.7173,  0.5408],
         [ 0.4470, -0.4491],
         [-0.2223,  0.7532],
         [-0.6478, -0.5310]], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0.], requires_grad=True)]

In [31]:
num_epochs = 200
for epoch in range(num_epochs):
    for X,Y in data_iter:
#         print(X,Y)
        l = loss(net(X), Y)
        trainer.zero_grad()
        l.backward()
        trainer.step() #optimizer
    l = loss(net(features), labels)
#     print(net[0].weight.data)
    if (epoch%10 == 0):
        print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.200671
epoch 11, loss 0.006001
epoch 21, loss 0.002899
epoch 31, loss 0.002154
epoch 41, loss 0.001544
epoch 51, loss 0.001318
epoch 61, loss 0.000999
epoch 71, loss 0.001042
epoch 81, loss 0.001461
epoch 91, loss 0.000862
epoch 101, loss 0.001274
epoch 111, loss 0.000681
epoch 121, loss 0.000654
epoch 131, loss 0.000694
epoch 141, loss 0.000631
epoch 151, loss 0.000670
epoch 161, loss 0.000588
epoch 171, loss 0.000710
epoch 181, loss 0.000576
epoch 191, loss 0.000542


In [27]:
# how our model represent a data sample like this: [0.1, 0.2, 0.3, 0.4]
x = torch.tensor([0.1, 0.2, 0.3, 0.4])
net(x)

tensor([0.0988, 0.2021, 0.3215, 0.4062], grad_fn=<AddBackward0>)

In [28]:
# That was just a simple example with linear relationship between colmns. 
# How about more complex relationships? Read on through next notebooks.